---
# Pandas String Functions & Regular expressions
Pandas String Funcitons, Regex functions, and working with text

---

Code examples on the most frequently used functions - Collected, Created and Edited by Pawel Rosikiewicz www.SimpleAI.ch

__MOST IMPORTNAT LINK (all pandas str functions!) are here__     
https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html

### __What so special in text data?__
* often messy
* usually not in a standardized format ready for analysis
* more difficult to make solid conclusions, or plot,

### __String functions in pandas vs built-in string functions__
* examples of built-in string functions are in Python_data_structures notebook
* Python's string functions are for individual string objects
* Pandas functions are for Series and DataFrames

### __Pandas string funcitons__
* str attribute    
* names matching the corresponding build-in string functions         

In [80]:
import re
import numpy as np
import pandas as pd

## CREATED & INSPECT
---

![outliers_slide_01](images/Regex_and_String_Functions__Slide1.png)

### Create raw string & pd.Series with substrings

In [55]:
# string example
''' pd.series, different dtypes, each item is called a substring
    string buildin, funcitons, works only on each item separately, 
'''
s = pd.Series(['0','PawelBig', 'Anna Romanow', '01236789', np.nan, ' '])

In [56]:
# raw string example
'''used to search patterns with regex'''
pat = r'[a-z]+'

### Inspect substrings

In [58]:
s.dtype # dtype('0') for mixed dtypes

dtype('O')

In [32]:
# inspect each substring
s.str.len() # len of each substrings in a series
s.str.isalpha() # np.nan returns NAN!
s.str.isnumeric() # '0' & '01236789' are numeric!
s.str.isalnum() # if yoiu have even one whitespace - is is false
s.str.isdigit() # eas expected, 
s.str.isspace() # True for strings wiht empty space only ' '
s.str.islower() # True only if all char are ...
s.str.isupper() # same ...
s.str.isdecimal() # if all char are decimal, 
s.str.istitle() # This Is Title

0    False
1    False
2     True
3    False
4      NaN
5    False
dtype: object

---
## SEACRH & FIND
---

* you search for pattern in a string, 
* can use regular expression that are at the end of this notebook


![outliers_slide_01](images/Regex_and_String_Functions__Slide4.png)

### Find pattern
ie. if len(series)==8, then there will be 8 results retiurnsed in Pd.Series   
__.str functions may return for each substring:__
* True/False
    * __str.contains__; pat='REGEX', flags=re.IGNORECASE, na="place for NaN", case=True - case sensitive, regex=True, if False, it searches string literal, can be used with & | (or) eg. s.str.contains('0')|s.str.contains('1'), s.str.contains('0|1')
    * __str.match__; no case, and flas parameters, 
    * __str.startwith__; -||-
    * __str.endwith__; -||-
    
    </br>
    
* Index nr in each substring
    * __str.find__; the lowest index, with entire pattern contained in a substring after it.
    * __str.rfind__; the highes index ....
    * __str.index__; values error if not found, 
    * __str.rindex__; ...
        
    </br>
    
* 0/1
    * __str.get_dummies__; in different columns, 0/1

### (a) Count
---

In [ ]:
s = pd.Series(['0','PawelBig', 'Anna Romanow', '01236789', np.nan, ' '])

In [313]:
# count, ....................................................
"""flags, yes, no na fill"""
s.str.count("0", flags=re.IGNORECASE).values

array([ 1.,  0.,  0.,  1., nan,  0.])

### (b) fidn if is present - True/False: 
Contains, match, startwith, endwith

---

In [305]:
s = pd.Series(['0','PawelBig', 'Anna Romanow', '01236789', np.nan, ' '])

In [108]:
# contains, ....................................................
"""ignorcase, from re module - only strings as inputs"""
s.str.contains(pat='big', flags=re.IGNORECASE).values.tolist()

[False, True, False, False, nan, False]

In [109]:
# select
"""remeber to replace na if you use that for selection!"""
flag = s.str.contains(pat='big', flags=re.IGNORECASE, na=False).values.tolist()
s.iloc[flag]

1    PawelBig
dtype: object

In [76]:
# more options
s.str.contains(
    pat='Pa', 
    case=True,# case sensitive
    regex=True, # if False, treats patern, as string literal, no regex
    na="whatever_here is Na" # Fill value for Na
).values

array([False, True, False, False, 'whatever_here is Na', False],
      dtype=object)

In [85]:
# and, or 
s.str.contains('0') | s.str.contains('1') #  0 or 1
s.str.contains('0') & s.str.contains('1') #  0 and 1
s.str.contains('0|1').values # works only with |

array([True, False, False, True, nan, False], dtype=object)

#### --- match, stre/endwith 
* match is more strinct then contains, 
* starte/end with are always casesensitive!
    * no flags!

In [314]:
# match, ....................................................
s.str.match(pat='Pa', flags=re.IGNORECASE).values

array([False, True, False, False, nan, False], dtype=object)

In [93]:
# stars/endswith, ....................................................
s.str.startswith(pat='Paw', na="no other pasrams").values
s.str.endswith(pat='now', na="no other pasrams").values

array([False, False, True, False, 'no other pasrams', False], dtype=object)

### (c) find pattern idx 
find, rfind, index, rindex

---
* find & rfind, retuirn pattern idx in each substring
    * Each of returned indexes corresponds to the position where the substring is fully contained between [start:end].
    * Return -1 on failure.
    * differences:
        * find - first encountered substring
        * rfind - the last encourntered substring
* index & rindex; works the same, but retunr ValiueError if pattern is not found

In [136]:
s = pd.Series(['0','PawelBig', 'Anna Romanow', '012367089', np.nan, ' '])

In [315]:
# find, ....................................................
"""find idx with the lowest value"""
s.str.find(sub="0").values

array([ 0., -1., -1.,  0., nan, -1.])

In [129]:
# search from to
s.str.find(sub="0", start=0, end=3).values

array([ 0., -1., -1.,  0., nan, -1.])

In [316]:
# rfind, ....................................................
"""idx with the highest value"""
s.str.rfind(sub="0").values

array([ 0., -1., -1.,  0., nan, -1.])

### (d) Get_dummies
---
* intersting option
    * Each string in Series is split by sep and returned as a DataFrame of dummy/indicator variables.
* do not return columns with nan presence,

In [317]:
# str.get_dummies, ....................................................
s.str.get_dummies()

,,0,01236789,Anna Romanow,PawelBig
0,0,1,0,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,0,0,1,0,0
4,0,0,0,0,0
5,1,0,0,0,0


In [318]:
pd.Series(['a|b', 'a', 'a|c']).str.get_dummies()

,a,b,c
0,1,1,0
1,1,0,0
2,1,0,1


### Find pattern and return it
findall, extract, extractall

---
* __MAin Function__
    * __.str.findall()__; returns pd.Series, with list in each cell
    * __.str.extract()__; extract first match only, NAN IF NOTHING,expand=True, possible
    * __.str.extractall()__; extratc all mathcing patterns, if>1, creats mutiindex, always expanding results to df
* __Notes__:
    * these functions, match the pattern and extract it for work, or examination,
    * the rest of the substring is ignored

In [320]:
s = pd.Series(['bla', 'BLA', 'some random BLA and bla','blablabla'])

In [327]:
# findall, ....................................................
'''return series with lists in each cell'''
s.str.findall('bla', flags=re.IGNORECASE)

0              [bla]
1              [BLA]
2         [BLA, bla]
3    [bla, bla, bla]
dtype: object

In [322]:
# search only the end, or start
s.str.findall('bla$', flags=re.IGNORECASE) # endswith
s.str.findall('^bla', flags=re.IGNORECASE) # startswith, but with flags:)

0    [bla]
1    [BLA]
2       []
3    [bla]
dtype: object

#### extract and extractall functions
* require capture string in brackets eg: pat=r'(bla)'

In [325]:
# extract, ....................................................
'''returns only the first instance'''
s.str.extract(
    pat=r'(bla)', 
    expand=False, 
    flags=re.IGNORECASE
).values

array(['bla', 'BLA', 'BLA', 'bla'], dtype=object)

In [329]:
# extractall, ....................................................
'''extracts all matchin patterns - created df with multiindex'''
s.str.extractall(
    pat=r'(bla)', 
    flags=re.IGNORECASE
)# caution on mutiindex, 

0
  match     
0 0      bla
1 0      BLA
2 0      BLA
  1      bla
3 0      bla
  1      bla
  2      bla

---
## SIZE/LENGHT TRANSFORMATIONS
---

![outliers_slide_01](images/Regex_and_String_Functions__Slide2.png)

### (a) SLICE
with idx, str.slice, str.get

---

* __with idx__; [idx] or [from, to], 
* __str.slice__; (from, to)
* __.str.get(one_idx_only)__; only one idx can e 
* Coments:
    * if we use from, to in any method, it returns, a sunstring, or its max lenght,
    * if we use only one index nr: it will return NaN if that index exceeds the lenghts of a substring
    * it works the ssame with all fucntions,


In [331]:
s = pd.Series(['bla', 'BLA', 'some random BLA and bla','blablabla', np.nan])

In [193]:
# slice with the index, ....................................................
s.str[5].values # NaN if idx>len(string)
s.str[0:20].values # eturns max char

array([nan, nan, 'r', 'a', nan], dtype=object)

In [330]:
# slice from to, .................................................... 
s.str.slice(0,20).values # same as str[0:20]

array(['bla', 'BLA', 'some random BLA and ', 'blablabla'], dtype=object)

In [202]:
# str.get(), ....................................................
'''only one char'''
s.str.get(5).values
s.str.get(-1).values

array(['a', 'A', 'a', 'a', nan], dtype=object)

#### (b) JOIN & CONCATENATE
elements in 1 series, 2 series, and series with different lenght and indexes - str.cat(), 
__.str.cat(join={'inner', 'outer', 'left', 'right'})__

---


In [203]:
s1 = pd.Series(["a", "b", "c", np.nan])
s2 = pd.Series(["1", "2", np.nan, "4"])

In [333]:
# join for strings, ...............................................
";".join(s1.values.tolist())

'a;b;c'

In [334]:
# str.cat(), ....................................................
'''Elements in 1 series'''
s1.str.cat() # 'abc'
s1.str.cat(sep=";_") # 'a;_b;_c'
s1.str.cat(sep=";_", na_rep="...") # 'a;_b;_c;_...'

'a;_b;_c'

In [336]:
# join items in two strings
''' Elements in 3 series
    ...
    CAUTION: 1. best if the have the same lenght & indexing
            2. if na_rep is not used - it returns NaN, 
               for any joints with missing data
'''
s1.str.cat(s2, na_rep="_blah_")

0    a_blah_
1         b1
2         c2
dtype: object

#### join string with different lenghts and indexes
* __.str.cat(join={'inner', 'outer', 'left', 'right'})__

In [223]:
s1 = pd.Series(["a", "b", "c"])
s2 = pd.Series([str(x) for x in list(range(1,5))], index=[1,2,6,7])

In [230]:
# default
'''join, as in left'''
s1.str.cat(s2, sep=",")

0    NaN
1    b,1
2    c,2
dtype: object

In [225]:
s1.str.cat(s2, join="outer", na_rep="----", sep=",")# use all avialbe indexes, 

0    a,----
1       b,1
2       c,2
6    ----,3
7    ----,4
dtype: object

In [229]:
s1.str.cat(s2, join="inner", na_rep="----", sep=",")
s1.str.cat(s2, join="left", na_rep="----", sep=",") # usee all from left, 
s1.str.cat(s2, join="right", na_rep="----", sep=",") # usee all from right, 

1       b,1
2       c,2
6    ----,3
7    ----,4
dtype: object

### (c) Divide substrings - return 2 ITEMS - before, and after items
str.split, str.partition and str.rpartition

---


In [232]:
s = pd.Series(['bla', 'some random BLA and bla','blablabla', np.nan])

In [337]:
# split , ....................................................
'''default: by white spaces'''
s.str.split(expand=True)

,0,1,2,3,4
0,bla,None,None,None,None
1,BLA,None,None,None,None
2,some,random,BLA,and,bla
3,blablabla,None,None,None,None
4,NaN,NaN,NaN,NaN,NaN


In [250]:
# str.split with max_nr of splits, 
'''no flags, 
   Caution, n+1 for max splits, its index nr!
'''
s.str.split(
    pat="l",
    n=2, # max nr of splits, +1, because uts index nr. not lenght
    expand=True)

,0,1,2
0,b,a,None
1,some random BLA and b,a,None
2,b,ab,abla
3,NaN,NaN,NaN


###  (c) Divide substrings - return 3 ITEMS
partiotion, and rpartition

---

In [241]:
s = pd.Series(['bla', 'some random BLA and bla','blablabla', np.nan])

In [340]:
# partion, ...................................................
'''split on the FIRTS pattern occcurence'''
s.str.partition("l", expand=True)

,0,1,2
0,b,l,a
1,BLA,,
2,some random BLA and b,l,a
3,b,l,ablabla
4,NaN,NaN,NaN


In [339]:
# rpartiotion, ...................................................
'''split on the LAST pattern occcurence'''
s.str.rpartition("l", expand=True)

,0,1,2
0,b,l,a
1,,,BLA
2,some random BLA and b,l,a
3,blablab,l,a
4,NaN,NaN,NaN


## CONTENT MODIFICATIONS
---

![outliers_slide_01](images/Regex_and_String_Functions__Slide3.png)

### __(A) REPLACE__
---
* __.str.replace__; s.str.replcase("from", "into"), case, n, 
* __.str.slice_replace__; takes str, indexes to remove, and replaces them with new pattern, no case parameter!

In [258]:
s = pd.Series(['bla', 'some random BLA and bla','blablabla', np.nan])

s.str.replace(
    pat="l", 
    repl="-MOCCA-",
    n=2, # Number of replacements to make from start
    case=False
)


0                              b-MOCCA-a
1    some random B-MOCCA-A and b-MOCCA-a
2                  b-MOCCA-ab-MOCCA-abla
3                                    NaN
dtype: object

In [264]:
s.str.slice_replace(0,5,"______")

0                      ______
1    ______random BLA and bla
2                  ______abla
3                         NaN
dtype: object

In [370]:
## REPLCACE with Function

# eg remove system form each word and replcace with .
s = pd.Series(["BioSystem", "CarSystem", "TruckSystem"])
s.str.replace('(\w+tem)', lambda x: x.groups()[0][0:3]) # you must work on groups:

# sedong approach, same effect
def f(x):
    return x.groups()[0][:3]
s.str.replace('(\w+tem)', f)
        #            0
        # 0     Monday
        # 1    Tuesday
        # 2  Wednesday 


# one more example


## REPLCACE with Function
s = pd.Series(["Buldog", "Maddog", "Catdog"])
s.str.replace('(\w{0,10}dog)', lambda x: x.groups()[0][0:3])  # you must work on group with lambda
        # 0    Bul
        # 1    Mad
        # 2    Cat

/Users/pawel/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/pawel/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.
/Users/pawel/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: The default value of regex will change from True to False in a future version.


0    Bul
1    Mad
2    Cat
dtype: object

### (B) CHANGE ENDS
---
* __Remove ends__
    * __.str.strip()__; both ends,s.str.strip(to_strip="Pattrern that will be removed"), no idx, or int accepted
    * __.str.rstrip()__; only end/right
    * __.str.lstrip()__; only beginning/left
* __Padding__
    * __.str.pad()__; 


In [277]:
s = pd.Series(['bla__', 'some random BLA and __bla','blabla__bla', np.nan])

In [276]:
# strip specified ends
''' you must provide pattern that will be removed
    CAUTION - doent work with indexes, or integers!, returns NaN only
'''
s.str.strip(to_strip="bla").values

array(['__', 'some random BLA and __', '__', nan], dtype=object)

In [287]:
# padding
''' . widht; min width of resulting string; 
            additional characters will be filled with fillchar
    . fillchar; “string of my choice”
    . side;  “both”, “right”, “left”
    comments:
        - NaN are not used, 
        - filchar, must be a sincle character, not a string
'''
s.str.pad(
    width=30,
    fillchar=":", # MUST BE A SINGLE CHARACTER
    side="both"
)

0    ::::::::::::bla__:::::::::::::
1    ::some random BLA and __bla:::
2    :::::::::blabla__bla::::::::::
3                               NaN
dtype: object

### (C)  CHNAGE COPY NR
---

In [289]:
s = pd.Series(['bla__', 'some random BLA and __bla','blabla__bla', np.nan])

In [290]:
s.str.repeat(2)

0                                           bla__bla__
1    some random BLA and __blasome random BLA and _...
2                               blabla__blablabla__bla
3                                                  NaN
dtype: object

In [293]:
pd.Series(['a', 'b', 'c']).str.repeat(repeats=[1,2,3])

0      a
1     bb
2    ccc
dtype: object

### (D)  LOWER/UPPER CASE
---

In [295]:
s = pd.Series(['bla__', 'some random BLA and __bla','blabla__bla', np.nan])

In [304]:
s.str.lower()
s.str.upper()
s.str.title()
s.str.capitalize() # upper only for the first char.
s.str.swapcase()
s.str.casefold() # remove any distintion of case in str, - all are lower now

0                        bla__
1    some random bla and __bla
2                  blabla__bla
3                          NaN
dtype: object

---
## PART 2 - REGEX
---

![outliers_slide_01](images/Regex_and_String_Functions__Slide5.png)

In [343]:
import re
import numpy as np
import pandas as pd

In [342]:
#### |, &   - OR , AND .....................
s=pd.Series(['0', 'John Wood', 'Colin Welsh', 'my list', 
             '02456', np.nan, 'HELLO WORLD', 'water%'])
s.str.contains('John') | s.str.contains('Colin') # or
s.str.contains('John|Colin').values # same results

array([False, True, True, False, False, nan, False, False], dtype=object)

In [345]:
#### . -<dot>,  .....................
#
#.         matches any ONE CHARACTER except for new line
s2 = pd.Series(['bAR', 'sugAR', 'c\nARtoon', 'ARgon'])
s2.str.contains('.AR', flags=re.IGNORECASE).values
        # array([ True,  True, False, False])

array([ True,  True, False, False])

In [361]:
####  SET of CHARACTERS  _________________________________________________
#.
#.      []   match any char in a set
#.      [^]  -||- no in a set; ^has to be at the beginning!
#.         ...
#.         [a-z] - match any lowercase letter
#.         [A-Z] - match any uppercase letter
#.         [0-9] - match any digit
#.         [a-zA-Z0-9] - match any letter or digit
#.         ...

s = pd.Series(['mop', 'top', 'gluonop', 'sky9op', 'op', np.nan])

# mach any char in a set
print(1, s.str.contains( '[mn9]op' ).values) # matches whatever contains m,n or 9
print(2, s.str.contains( '[a-z]op' ).values) # True, if at least one lover case letter is found
print(3, s.str.contains( '[a-zA-Z0-9]op', na=False ).values) # whatever before op, except ""
print(4, s.str.contains( '.op', na=False ).values) # alternative, with dot "."
print(5, s.str.contains( '[a-z]{1}op', na=False ).values) # here we ask for only one char


# match any char NOT in that set
print(4, s.str.contains( '[^a-z]op', na=False ).values)
        #.   array([ True,  True, False, False])

1 [True False True True False nan]
2 [True True True False False nan]
3 [ True  True  True  True False False]
4 [ True  True  True  True False False]
5 [ True  True  True False False False]
4 [False False False  True False False]


In [364]:

#### - Types of Characters  _________________________________________________
#
#.         \d - c
#.         \D - match any non digit
#.         \w - match a word character
#.         \W - match a non-word character
#.         \s - match whitespace (spaces, tabs, newlines, etc.)
#.         \S - match non-whitespace

s = pd.Series(['mop999', 'top', 'gluonop', 'sky9op',"  ", np.nan])

# digit / not a digit (d/D)
print(1, s.str.contains('[\d]', na=False).values)  # any digit, in any number
print(2, s.str.contains('[\D]', na=False).values,"\n")
        # 1 [ True False False  True False False]
        # 2 [ True  True  True  True  True False] 

# world / not a world (w/W)
print(3, s.str.contains('\w', na=False).values)
print(4, s.str.contains('[\W]', na=False).values,"\n")
        # 3 [ True  True  True  True False]
        # 4 [False False False  True False]

# whitespace / not a whitespace (s/S)
print(5, s.str.contains('[\s]', na=False).values)
print(6, s.str.contains('[\S]', na=False).values,"\n")
        # 5 [False False False False  True False]
        # 6 [ True  True  True  True False False]

# same without brackets
print(7, s.str.contains('\dop', na=False).values)
        # [False False False  True False False]



    

1 [ True False False  True False False]
2 [ True  True  True  True  True False] 

3 [ True  True  True  True False False]
4 [False False False False  True False] 

5 [False False False False  True False]
6 [ True  True  True  True False False] 

7 [False False False  True False False]


In [365]:
#### - POSITION IN STRING  _________________________________________________ 
#
#.        ^ - match at the beginning of a string, Caution, not the same as [^]
#,            not beginning of the world
#.        $ - searches for matches at the end of a string

s = pd.Series(['mop', 'topmmmmm65', 'gluonop', 'sky9op',"  ", np.nan])

# beginning of the line
print(1, s.str.contains('^m', na=False).values,"\n") # BEGINNING 

# end of the line
print(2, s.str.contains('[\d]$', na=False).values,"\n") # END $




1 [ True False False False False False] 

2 [False  True False False False False] 



In [367]:
#### - COPY NUMBER   _________________________________________________
#.          
#.        *, +, ?, {}, - writen behind a given char/substring
#         order.       - char muct be in the same order, the one between must also be included
#
#.  General:
#.        * - match ZERO or MORE copies ;   ≥0    copies
#.        ? - match ZEOR or ONE copy    ;   [0,1] copies
#.        + - match ONE  or MORE copies ;   ≥1    copies
#.
#.  Speciffic copy nr:
#.        {m} - match m copies of a given char
#.        {m,} - match m OR MORE copies of a given char
#.        {m,n} - match between m and n copies of a given char

s = pd.Series(['mop', 'topmmmmm65', 'mamgluopnop', 'sky9op',"  ", np.nan])

# beginning of the line
print(1, s.str.contains('m*', na=False).values,"\n") # zero or more "m"
print(2, s.str.contains('m+[a-zA-Z0-9]*p{1,}', na=False).values,"\n") 
# one or more "m" and then p, wiht aything between

1 [ True  True  True  True  True False] 

2 [ True False  True False False False] 



In [368]:
#### ESCAPE CHARACTER  _________________________________________________
#.          
#.        / - ALLOWS SEARCHING FOR OTHER SPECIAL CHARACTERS!
# .           eg: ., ^, +, *, ?, $, ...

s = pd.Series(["is this my chair?","yes+)","no:{}"])

print(1, s.str.contains(r'\?', na=False).values,"\n")
print(2, s.str.contains(r'yes\+\)', na=False).values,"\n")

1 [ True False False] 

2 [False  True False] 



In [369]:
#### GROUPING  _________________________________________________
#.          
#.        () - parenthesis define a subtring with REGEX in it
# .            very usefull for pd.str.extract()
#              that allows extracting certain substring and placing it in a Dataframe

s = pd.Series(["My loundry is at Monday and MoNNday only, but Mday is ok", 
               "Tuesday i sleep,because its MYTday", 
               "Every Wednesday I have Wedayparty"])


## EXTRACT:
#.           extract works only when using ()

# extract first match and put into df
print(s.str.extract(r'(\w+day)', expand=True),"\n")

# extract all and unstack for df
df = s.str.extractall('(\w+day)') # df with MultiIndex - can be unstack
df.unstack()




## REPLCACE with Function

# eg remove system form each word and replcace with .
s = pd.Series(["BioSystem", "CarSystem", "TruckSystem"])
s.str.replace('(\w+tem)', lambda x: x.groups()[0][0:3]) # you must work on groups:

# sedong approach, same effect
def f(x):
    return x.groups()[0][:3]
s.str.replace('(\w+tem)', f)
        #            0
        # 0     Monday
        # 1    Tuesday
        # 2  Wednesday 


# one more example


## REPLCACE with Function
s = pd.Series(["Buldog", "Maddog", "Catdog"])
s.str.replace('(\w{0,10}dog)', lambda x: x.groups()[0][0:3])  # you must work on group with lambda
        # 0    Bul
        # 1    Mad
        # 2    Cat

           0
0     Monday
1    Tuesday
2  Wednesday 



/Users/pawel/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/pawel/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:34: FutureWarning: The default value of regex will change from True to False in a future version.
/Users/pawel/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:46: FutureWarning: The default value of regex will change from True to False in a future version.


0    Bul
1    Mad
2    Cat
dtype: object